# Omdena TrashOut Project

This notebook will consist of the EDA on the data provided by TrashOut on the dumpsites all over the world (separated by continents).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#@title Required libraries being imported
"""
required libraries imported
"""

! pip install category_encoders
! pip install geopandas

from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

import category_encoders as ce

from __future__ import print_function
import os, io, sys, random, time, pprint
import numpy as np
from numpy import save, load
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import one_hot
from tensorflow.keras.callbacks import LambdaCallback, Callback, ModelCheckpoint
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.models import load_model
from tensorflow.keras import Model, Sequential
from tensorflow.keras import backend
from tensorflow.keras.layers import Activation, Reshape, Dense, Embedding, Dropout, Input, BatchNormalization, concatenate, Flatten, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Adadelta, Adagrad, Adamax
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical, plot_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

     |████████████████████████████████| 81kB 3.5MB/s 
     |████████████████████████████████| 962kB 4.7MB/s 
     |████████████████████████████████| 14.7MB 308kB/s 
     |████████████████████████████████| 10.9MB 42.6MB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
"""
relative paths for the different data files
"""

path = "/content/drive/My Drive/omdena/extraneous_data/"

africa0 = path + "africa_0.csv"
africa1 = path + "africa_1.csv"
asia = path + "asia_0"
europe0 = path + "europe_0"
europe1 = path + "europe_1"
europe2 = path + "europe_2.csv"
europe3 = path + "europe_3.csv"
europe4 = path + "europe_4.csv"
europe5 = path + "europe_5.csv"
europe6 = path + "europe_6.csv"
europe7 = path + "europe_7.csv"
europe8 = path + "europe_8.csv"
northamerica0 = path + "northamerica_0"
northamerica1 = path + "northamerica_1"
southamerica = path + "southamerica_0"
australia = path + "australia_0"
oceania = path + "oceania_0"

In [ ]:
"""
example data
"""

data = pd.read_csv(asia, sep=';')
data.head()

,id,latitude,longitude,continent,country,aa1,aa2,aa3,locality,subLocality,types,updateTime,updateNeeded,note,url,status,size,anonymous,created,accessibility,image
0,57823,32.354964,35.019643,Asia,Israel,Center District,HaSharon,NaN,Bahan,Bahan,domestic,2020-06-22T07:10:14.884Z,False,NaN,https://admin.trashout.ngo/trash-management/de...,cleaned,bag,False,2020-06-22T06:25:01.411Z,NaN,https://firebasestorage.googleapis.com/v0/b/tr...
1,57814,34.929792,32.862968,Asia,Cyprus,Limassol,NaN,NaN,NaN,Limassol,"deadAnimals,plastic",2020-06-21T11:17:53.751Z,False,NaN,https://admin.trashout.ngo/trash-management/de...,stillHere,bag,False,2020-06-21T11:17:53.751Z,notForGeneralCleanup,https://firebasestorage.googleapis.com/v0/b/tr...
2,57758,40.235476,44.505753,Asia,Armenia,Kotayk Province,NaN,NaN,Zovuni,Zovuni,metal,2020-06-15T10:08:23.046Z,False,NaN,https://admin.trashout.ngo/trash-management/de...,stillHere,wheelbarrow,False,2020-06-15T10:08:23.046Z,NaN,https://firebasestorage.googleapis.com/v0/b/tr...
3,57594,-6.119567,106.661225,Asia,Indonesia,Banten,Kota Tangerang,Benda,NaN,RW 010,"electronic,glass,metal,plastic,domestic",2020-06-04T12:22:35.542Z,False,NaN,https://admin.trashout.ngo/trash-management/de...,stillHere,car,False,2020-06-04T12:22:35.542Z,byCar,https://firebasestorage.googleapis.com/v0/b/tr...
4,57585,-2.677623,118.889706,Asia,Indonesia,Sulawesi Barat,Kabupaten Mamuju,Kecamatan Mamuju,NaN,Binanga,"glass,metal,plastic,domestic",2020-06-02T22:06:37.892Z,False,NaN,https://admin.trashout.ngo/trash-management/de...,stillHere,car,False,2020-06-02T22:06:37.892Z,byCar,https://firebasestorage.googleapis.com/v0/b/tr...


In [ ]:
"""
combining all data continent wise
"""

africa = [africa0, africa1]

europe = [europe0, europe1, europe2, europe3, europe4,
          europe5, europe6, europe7, europe8]

northamerica = [northamerica0, northamerica1]


africa = pd.concat([pd.read_csv(f, error_bad_lines=False, sep=';') for f in africa])
africa.to_csv(f'/content/drive/My Drive/omdena/africa.csv')

europe = pd.concat([pd.read_csv(f, error_bad_lines=False, sep=';') for f in europe])
europe.to_csv(f'/content/drive/My Drive/omdena/europe.csv')

northamerica = pd.concat([pd.read_csv(f, error_bad_lines=False, sep=';') for f in northamerica])
northamerica.to_csv(f'/content/drive/My Drive/omdena/northamerica.csv')

asia = pd.read_csv("/content/drive/My Drive/omdena/asia.csv", sep=';')
southamerica = pd.read_csv("/content/drive/My Drive/omdena/southamerica.csv", sep=';')
australia = pd.read_csv("/content/drive/My Drive/omdena/australia.csv", sep=';')
oceania = pd.read_csv("/content/drive/My Drive/omdena/oceania.csv", sep=';')

## By this point all the continents have one pandas dataframe each ready to be used.

In [ ]:
africa.head()

,id,latitude,longitude,continent,country,aa1,aa2,aa3,locality,subLocality,types,updateTime,updateNeeded,note,url,status,size,anonymous,created,accessibility,image
0,57838,-26.198062,28.051973,Africa,South Africa,Gauteng,City of Johannesburg Metropolitan Municipality,NaN,Johannesburg,Hillbrow,"deadAnimals,liquid,metal,organic,plastic,domestic",2020-06-23T10:10:44.056Z,False,NaN,https://admin.trashout.ngo/trash-management/de...,stillHere,car,False,2020-06-23T10:10:44.056Z,NaN,https://firebasestorage.googleapis.com/v0/b/tr...
1,57837,-26.196481,28.050331,Africa,South Africa,Gauteng,City of Johannesburg Metropolitan Municipality,NaN,Johannesburg,Hillbrow,"liquid,metal,plastic,construction,domestic",2020-06-23T10:04:35.937Z,False,NaN,https://admin.trashout.ngo/trash-management/de...,stillHere,car,False,2020-06-23T10:04:35.937Z,NaN,https://firebasestorage.googleapis.com/v0/b/tr...
2,57836,-26.182135,27.989881,Africa,South Africa,Gauteng,City of Johannesburg Metropolitan Municipality,NaN,Johannesburg,Westdene,"glass,liquid,metal,organic,plastic,domestic",2020-06-23T09:46:03.305Z,False,"The location is in Ashanti Street, Coronationv...",https://admin.trashout.ngo/trash-management/de...,stillHere,car,True,2020-06-23T09:46:03.305Z,byCar,https://firebasestorage.googleapis.com/v0/b/tr...
3,57831,-28.466159,28.855491,Africa,South Africa,Free State,Thabo Mofutsanyane,NaN,Phuthaditjhaba,Phuthaditjhaba,"glass,deadAnimals,dangerous,metal,organic,plas...",2020-06-22T14:42:45.226Z,False,NaN,https://admin.trashout.ngo/trash-management/de...,stillHere,car,True,2020-06-22T14:42:45.226Z,byCar,https://firebasestorage.googleapis.com/v0/b/tr...
4,57830,-28.514575,28.820170,Africa,South Africa,Free State,Thabo Mofutsanyane,NaN,Phuthaditjhaba,Phuthaditjhaba-A,"glass,dangerous,metal,organic,plastic,construc...",2020-06-22T13:08:51.901Z,False,NaN,https://admin.trashout.ngo/trash-management/de...,stillHere,car,False,2020-06-22T13:08:51.901Z,notForGeneralCleanup,https://firebasestorage.googleapis.com/v0/b/tr...


## Experimenting some data analysis with the Africa dataset.

In [ ]:
africa.shape

(5324, 21)

In [ ]:
africa.describe(include='all')

,id,latitude,longitude,continent,country,aa1,aa2,aa3,locality,subLocality,types,updateTime,updateNeeded,note,url,status,size,anonymous,created,accessibility,image
count,5324.000000,5324.000000,5324.000000,5324,5324,5276,3493,54,4151,4934,5324,5324,5324,558,5324,5324,5324,5324,5324,4100,5315
unique,NaN,NaN,NaN,1,40,147,195,22,266,497,595,5322,2,531,5324,4,3,2,5324,13,5315
top,NaN,NaN,NaN,Africa,Mozambique,Maputo,Matola,Malindi,Maputo,Matola,"glass,metal,organic,plastic,domestic",2016-11-08T18:17:09.000Z,True,it is nearby,https://admin.trashout.ngo/trash-management/de...,stillHere,car,False,2019-08-27T07:11:45.741Z,byCar,https://firebasestorage.googleapis.com/v0/b/tr...
freq,NaN,NaN,NaN,5324,4016,1756,675,15,1415,530,390,2,5254,5,1,4160,2387,4946,1,3542,1
mean,48949.304658,-18.359550,31.255820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,5756.230601,11.423127,12.652332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1306.000000,-34.123890,-17.435456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,48402.750000,-25.937456,32.482933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,49920.500000,-24.701592,32.592974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,51393.250000,-15.098122,35.345961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# aa3 has 5270 out of 5324 values null. Might as well be dropped as suggested by Rosana.
africa.isnull().sum()

id                  0
latitude            0
longitude           0
continent           0
country             0
aa1                48
aa2              1831
aa3              5270
locality         1173
subLocality       390
types               0
updateTime          0
updateNeeded        0
note             4766
url                 0
status              0
size                0
anonymous           0
created             0
accessibility    1224
image               9
dtype: int64

In [ ]:
africa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5324 entries, 0 to 323
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             5324 non-null   int64  
 1   latitude       5324 non-null   float64
 2   longitude      5324 non-null   float64
 3   continent      5324 non-null   object 
 4   country        5324 non-null   object 
 5   aa1            5276 non-null   object 
 6   aa2            3493 non-null   object 
 7   aa3            54 non-null     object 
 8   locality       4151 non-null   object 
 9   subLocality    4934 non-null   object 
 10  types          5324 non-null   object 
 11  updateTime     5324 non-null   object 
 12  updateNeeded   5324 non-null   bool   
 13  note           558 non-null    object 
 14  url            5324 non-null   object 
 15  status         5324 non-null   object 
 16  size           5324 non-null   object 
 17  anonymous      5324 non-null   bool   
 18  created  

In [ ]:
"""
removing some unnecesarry columns
"""

africa.drop(['aa3', 'continent', 'id', 'url', 'image', 'note'], axis=1, inplace=True)

In [ ]:
africa.head()

,latitude,longitude,country,aa1,aa2,locality,subLocality,types,updateTime,updateNeeded,status,size,anonymous,created,accessibility
0,-26.198062,28.051973,South Africa,Gauteng,City of Johannesburg Metropolitan Municipality,Johannesburg,Hillbrow,"deadAnimals,liquid,metal,organic,plastic,domestic",2020-06-23T10:10:44.056Z,False,stillHere,car,False,2020-06-23T10:10:44.056Z,NaN
1,-26.196481,28.050331,South Africa,Gauteng,City of Johannesburg Metropolitan Municipality,Johannesburg,Hillbrow,"liquid,metal,plastic,construction,domestic",2020-06-23T10:04:35.937Z,False,stillHere,car,False,2020-06-23T10:04:35.937Z,NaN
2,-26.182135,27.989881,South Africa,Gauteng,City of Johannesburg Metropolitan Municipality,Johannesburg,Westdene,"glass,liquid,metal,organic,plastic,domestic",2020-06-23T09:46:03.305Z,False,stillHere,car,True,2020-06-23T09:46:03.305Z,byCar
3,-28.466159,28.855491,South Africa,Free State,Thabo Mofutsanyane,Phuthaditjhaba,Phuthaditjhaba,"glass,deadAnimals,dangerous,metal,organic,plas...",2020-06-22T14:42:45.226Z,False,stillHere,car,True,2020-06-22T14:42:45.226Z,byCar
4,-28.514575,28.820170,South Africa,Free State,Thabo Mofutsanyane,Phuthaditjhaba,Phuthaditjhaba-A,"glass,dangerous,metal,organic,plastic,construc...",2020-06-22T13:08:51.901Z,False,stillHere,car,False,2020-06-22T13:08:51.901Z,notForGeneralCleanup
